In [1]:
import pyprind
import pandas as pd
import os 

basepath = "./data/aclImdb"

labels = {"pos": 1, "neg": 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()

for s in ("test", "train"):
    for l in ("pos", "neg"):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), "r", encoding="utf-8") as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
df.columns = ["review", "sentiment"]

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:56


In [2]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

df.to_csv("./movie_data.csv", index=False)

df = pd.read_csv("./movie_data.csv")
df.head(3)

,review,sentiment
0,Who ARE the people that star in this thing? Ne...,1
1,"Really, average is the only word that comes to...",0
2,First off... I never considered myself an Uwe ...,0


In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array(["The sun is shining", 
                 "The weather is sweet", 
                 "The sun is shining the weather is sweet, and one and one is two"])
bag = count.fit_transform(docs)

In [4]:
print(count.vocabulary_)

{'sweet': 5, 'weather': 8, 'sun': 4, 'the': 6, 'is': 1, 'shining': 3, 'two': 7, 'one': 2, 'and': 0}


In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm="l2", smooth_idf=True)
np.set_printoptions(precision=2)

print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]


In [6]:
def tokenizer(text):
    return text.split()

tokenizer("runner like running and thus they run")

['runner', 'like', 'running', 'and', 'thus', 'they', 'run']

In [7]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter("runner like running and thus they run")

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [8]:
import nltk

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from nltk.corpus import stopwords

stop = stopwords.words("english")
[w for w in tokenizer_porter("a runner likes running and runs a lot")[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# Training a logistic regression model for document classification

In [10]:
X_train = df.loc[:25000, "review"].values
y_train = df.loc[:25000, "sentiment"].values
X_test = df.loc[25000:, "review"].values
y_test = df.loc[25000:, "sentiment"].values

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [12]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:   18.3s


JoblibIndexError: JoblibIndexError
___________________________________________________________________________
Multiprocessing exception:
    ...........................................................................
/usr/lib/python3.4/runpy.py in _run_module_as_main(mod_name='IPython.kernel.__main__', alter_argv=1)
    165         sys.exit(msg)
    166     main_globals = sys.modules["__main__"].__dict__
    167     if alter_argv:
    168         sys.argv[0] = mod_spec.origin
    169     return _run_code(code, main_globals, None,
--> 170                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='IPython.kernel.__main__', loade...hon3.4/dist-packages/IPython/kernel/__main__.py')
    171 
    172 def run_module(mod_name, init_globals=None,
    173                run_name=None, alter_sys=False):
    174     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.4/runpy.py in _run_co

In [13]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score

np.random.seed(0)
np.set_printoptions(precision=6)
y = [np.random.randint(3) for i in range(25)]
X = (y + np.random.randn(25)).reshape(-1, 1)

cv5_idx = list(StratifiedKFold(y, n_folds=5, shuffle=False, random_state=0))

cross_val_score(LogisticRegression(random_state=123), X, y, cv=cv5_idx)

array([ 0.6,  0.4,  0.6,  0.2,  0.6])

In [14]:
from sklearn.grid_search import GridSearchCV

gs = GridSearchCV(LogisticRegression(), {}, cv=cv5_idx, verbose=3).fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ....................................... , score=0.600000 -   0.0s
[CV]  ................................................................
[CV] ....................................... , score=0.400000 -   0.0s
[CV]  ................................................................
[CV] ....................................... , score=0.600000 -   0.0s
[CV]  ................................................................
[CV] ....................................... , score=0.200000 -   0.0s
[CV]  ................................................................
[CV] ....................................... , score=0.600000 -   0.0s


[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [15]:
print(gs.best_score_)
print(cross_val_score(LogisticRegression(), X, y, cv=cv5_idx).mean())

0.48
0.48


In [16]:
import numpy as np
import re
from nltk.corpus import stopwords

def tokenizer(text):
    text = re.sub("<[^>]*>", "", text)
    emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text.lower())
    text = re.sub("[\W]+", " ", text.lower()) + " ".join(emoticons).replace("-", "")
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, "r", encoding="utf-8") as csv:
        next(csv)
        for line in csv:
            text , label = line[:-3], int(line[-2])
            yield text, label

In [17]:
next(stream_docs(path="./movie_data.csv"))

('"Who ARE the people that star in this thing? Never heard of them!! But this is one of the funniest comedies I have run across. It should win the Putz Puller Prize for Parody. The absurd starts with Dr. Jeykl snorting his powder and turning into a sex fiend.He is pursued by libido driven nurse early in the movie in one of the funniest scenes of the movie. Pay attention to the hospital PA system in the background; rather like the system in MASH. The final scene with Hyde accepting the award has had me laughing for years. Oh... and the ""Busty Nurse"" is Cassandra Peterson, who went on to become Elvira, Mistress of the Dark. <br /><br />If you liked the Mel Brooks classic movies (Blazing Saddles, etc.), I suspect you\'d like this one.<br /><br />Damn shame you can\'t get it on DVD anywhere.<br /><br />It\'s available on DVD now !!!!! Good thing DVDs don\'t wear out from use !!!!!"',
 1)

In [18]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
            return None, None
    return docs, y

In [19]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error="ignore",
                         n_features=2**21,
                         preprocessor=None,
                        tokenizer=tokenizer)

clf = SGDClassifier(loss="log", random_state=1, n_iter=1)
doc_stream = stream_docs(path="./movie_data.csv")

In [20]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:16


In [21]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print("Accuracy: %.3f" % clf.score(X_test, y_test))

Accuracy: 0.872


In [22]:
clf = clf.partial_fit(X_test, y_test)

In [27]:
import pickle
import os 

dest = os.path.join("movieclassifier", "pkl_objects")
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop, open(os.path.join(dest, "stopwords.pkl"), "wb"))
pickle.dump(clf, open(os.path.join(dest, "classifier.pkl"), "wb"), protocol=4)

In [30]:
import pickle
import re
import os
from vectorizer import vect
cl = pickle.load(open(os.path.join("movieclassifier", "pkl_objects", "classifier.pkl"), "rb"))

In [ ]:
import numpy as np
label = {0: "negative", 1:"positive"}
example = ["I love this movie"]
X = vect.transform(example)
print("Prediction: %s\nProbability: %.2f%%" %\(label[clf]))